In [22]:
import linsolve
import tf_linsolve
import scipy
import tensorflow as tf
import numpy as np
import pylab as plt
from copy import deepcopy
%load_ext line_profiler
%load_ext snakeviz

from hera_cal import utils
from hera_cal.redcal import RedundantCalibrator, get_reds, make_sol_finite, calibrate_in_place, redundantly_calibrate
from hera_sim.antpos import hex_array
from hera_sim.vis import sim_red_data

from hera_cal.redcal import redcal_iteration

from tqdm import tqdm
import itertools

from uvtools.utils import FFT

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [2]:
def powerspectra(data, taper='blackman-harris'):
    """
    """
    return np.abs(FFT(data, taper=taper))

## Realistic Example

In [23]:
f = np.load('../profiling/zen.2458098.27465.sum.corrupt.npz', allow_pickle=True)
data = dict(f['data'].tolist())
wgts = dict(f['wgts'].tolist())
freqs = f['freqs']
antpos = dict(f['antpos'].tolist())
reds = get_reds(antpos)

In [24]:
info_new = tf_linsolve.redcal.RedundantCalibrator(reds)
info = RedundantCalibrator(reds)

In [14]:
def firstcal_new(data, freqs, i=100):
    """
    """
    meta, sol0_new = info_new.firstcal(data, freqs, max_grps=i, min_vis_per_ant=None)
    
def firstcal(data, freqs):
    """
    """
    meta, sol0 = info.firstcal(data, freqs)

In [32]:
from tf_linsolve.redcal import fft_dly_new, interp_peak_new

In [52]:
def newrun(data, freqs, wgts):
    """
    """
    meta, sol0_new = info_new.firstcal(data, freqs, max_grps=10, min_vis_per_ant=None)
    meta, log_sol_new = info_new.logcal(data, sol0=sol0_new)
    meta, sol_new = info_new.omnical(data, log_sol_new, wgts=wgts, maxiter=500)
    return meta, sol0_new, log_sol_new, sol_new
    
def redcalrun(data, freqs, wgts):
    """
    """
    meta, sol0 = info.firstcal(data, freqs)
    meta, log_sol = info.logcal(data, sol0=sol0)
    meta, sol = info.omnical(data, log_sol, wgts=wgts, maxiter=500)

In [53]:
%%snakeviz -t
_ = newrun(data, freqs, wgts)

 
*** Profile stats marshalled to file '/var/folders/kv/bmmfw089255d48342x5bshcw0000gn/T/tmpzvkrltc2'. 
Opening SnakeViz in a new tab...


In [16]:
%%snakeviz -t
_ = firstcal(data, freqs)

 
*** Profile stats marshalled to file '/var/folders/kv/bmmfw089255d48342x5bshcw0000gn/T/tmptxqqrebw'. 
Opening SnakeViz in a new tab...


In [38]:
d = {k: v[:1, :] for k, v in data.items()} 

In [46]:
%lprun -f info_new._firstcal_iteration info_new._firstcal_iteration(d, np.diff(freqs)[0], freqs[0], max_grps=100)


In [47]:
%lprun -f info._firstcal_iteration info._firstcal_iteration(d, np.diff(freqs)[0], freqs[0])